# Tratos generales de Buen Código

## Tabla  de contenidos
***


***

El código es probablemente la representación más detallada del diseño. El objetivo final es hacer el código lo más robusto posible y escribirlo de tal manera que se pueda minimizar los defectos o hacerlos poco evidentes.

Software de buena calidad debería ser construido alrededor de los principios que se van a presentar a continuación, los cuales pueden servir como herramientas de diseño. Esto no significa que todos estos principios deban usarse siempre, pues algunos dependen del contexto y no son siempre aplicables.

## Diseño por contrato (Design by contract)

Algunas partes de nuestro software no fueron diseñadas para ser llamadas directamente por los usuarios, sino por otras partes del código. Tal es el caso en el que dividimos las responsabilidades de la aplicación en diferentes componentes o *layers* y tenemos que pensar en la interacción entre ellos.

Se tiene que encapsular cierta funcionalidad detras de cada componente y mostrar una interfaz a los clientes que van a usar esa funcionalidad, llamada, *Application Programming Interface (API)*. Las funciones, clases o métodos que escribimos para un componente en particular tienen una forma particular de trabajar bajo ciertas condiciones y si es que no se cumplen, el código se crashea.  

La idea detrás de DbC, es que en vez de poner implícitamente en el código que está esperando cada parte a recibir, ambas partes se ponen de acuerdo en un contrato, que si es violado, se va a levantar una excepción, diciendo claramente porque no puede continuar.

En este contexto, un contrato es una construción que fuerza a que algunas reglas se cumplan durante la comunicación de componentes de software. Hay precondiciones, postcondiciones, *invariants* y efectos colaterales.

- **Precondiciones**: Son los checks que el código va a hacer antes de correr. Va a chequear todas las condiciones que deben cumplirse antes de que la función se ejecute.
- **Postcondiciones**: Las validaciones son hechas luego de que el call de la función es retornada.
- **Invariants**: Son las cosas que se mantienen constantes mientras el código de la función se corre.
- **Efectos colaterales**: Podría ser bueno mencionar los efectos laterales en un docstring.

La razón por la que podríamos diseñar por contrato es que si ocurre un error, debe ser fácil de identificar así se puede corregir de forma rápida. Más importante, queremos que las partes críticas del código eviten ser ejecutadas bajo supuestos equivocados. 

Si la precondición falla, sabemos que es por culpa del usuario. Si la postcondición falla, es por un error en el código.

### Precondiciones

Las precondiciones son todas las garantías que un método o función espera recibir para así funcionar correctamente. Una función debería tener una validación adecuada para la información que va a manejar.

Surge la pregunta de en que lugar poner la validación. Una de las prácticas más comunes, es validar los datos una vez se han recibido todos, antes de ejecutar la lógica de la función. Generalmente este es la elección más segura en términos de robustez. Debemos tener en mente que la validación debe ser realizada solo por una de las partes del contrato, no ambos.

## Postcondiciones

Asumiendo que la función o método ha sido llamado con las propiedades correctas, las postcondiciones deben de garantizar que ciertas propiedades son conservadas. La idea es usar postcondiciones para checkear y validar todo lo que el cliente pueda necesitar.

## Diseño por contrato - Conclusiones

La idea principal de este principio de diseño es identificar en que parte se sitúa el problema. Definiendo un contrato, cuando algo falla en runtime, va a quedar muy claro que parte del código está rota, y lo que rompió el contrato. Es una buena idea implementar este principio para componentes críticos de la aplicación.

Para que este método sea efectivo, debemos pensar cuidadosamente que es lo que deseamos validar, y esto debe tener un valor significativo.

## Programación a la defensiva

Lo que se busca aquí es hacer que ciertas partes del código sean capaces de protegerse a sí mismas de inputs inválidos. Las ideas principales de la programación a la defensiva son como manejar errores para escenarios que creemos van a suceder y como lidiar con errores que no deberían ocurrir nunca (cuando ocurren condiciones imposibles). El primer caso se remonta a como manejar errores, mientras que el segundo caso se usan *assertions*.

### Error handling

La idea detrás de error handling es responder a los errores esperados en un intento de continuar con la ejecución de nuestro programa o decidir fallar si el error es insuperable.

#### Sustitución de valores

En algunos escenarios, cuando hay un error y hay riesgo de que el software produzca un valor incorrecto o falle completamente, podríamos reemplazar el resultado, con otro valor más seguro. Sin embargo, la sustitución de valores no es siempre posible. Esta estrategia debe ser cuidadosamente elegida para casos en los que el valor que se sustituye es una opción segura. Hacer esta decisión es un trade-off entre robustez y correctitud.

Si la aplicación es crítica, o los datos que se manejan son sensibles, esto no es una opción, dado que no podemos permitirnos el proveer a los usuarios resultados erróneos.

#### Manejo de excepciones

Si ocurre una falla en la función, esta debería de forma clara y no ambigua, notificar al resto de la aplicación respecto a los errores que no pueden ser ignorados así pueden ser tratados de forma adecuada.

El mecanismo para lograr lo anterior es una excepción. Es importante enfatizar que esto es para lo que se deberían utilizar las excepciones - para anunciar de forma clara una situación excepcional y no alterar el flujo del programa. Si el código intenta utilizar excepciones para manejar escenarios esperados, el flujo del programa se tornará más difícil de leer.

Las excepciones son acerca de notificar al *caller* acerca de algo que va mal. Esto significa que las excepciones deberían ser utilizadas cuidadosamente porque debilitan la encapsulación. Mientras más excepciones tenga una función, más cosas va a tener que anticipar el *caller*, por tanto debe de saber más acerca de la función que está llamando. Si una función tiene muchas excepciones, podría ser una señal de que podría ser desglosada en múltiples funciones más pequeñas.

#### Manejo de excepciones en el nivel correcto de abstracción
Las excepciones son una parte de las funciones que hacen solo una cosa, y solo una. La excepción que la función está manejando (o *raising*) tiene que ser consistente con la lógica en ella.

Las excepciones llevan consigo un significado. Por esta razón, es importante manejar cada tipo de excepción en el nivel correcto de abstracción. Dado que también pueden llevar información sensible de vez en cuando, no queremos que caiga en manos equivocadas.

#### No exponga tracebacks a usuarios finales
Esto es por consideraciones de seguridad. En Python, las excepciones de tracebacks contienen información muy rica y útil para debuggear. Desafortunadamente, esta información es muy útil para hackers o usuarios maliciosos que quieren probar y dañar la aplicación. Si usted elige que las excepciones se propaguen, asegúrese de no revelar información sensible. Si usted elige notificar a los usuarios sobre un problema, elija un mensaje genérico.

#### Evite bloques except vacíos
Lo peor que podría suceder es que un bloque de except use pass silenciosamente sin hacer nada.

In [1]:
def funcion():
    """Inserte lo que hace la función"""
    pass

try:
    funcion()
except:
    pass

Hay dos alternativas para lo anterior:
- Capturar una excepción más específica
- Realizar error handling en el bloque except

Lo mejor sería aplicar ambas recomendaciones. Manejar una excepción más específica va a ser el programa más mantenible porque el lector va a saber que esperar y va a tener una idea del porque de aquello.

Manejar la excepción puede significar muchas cosas. En la forma más simple, podría ser solamente loggear la excepción (asegúrese de usar logger.exception o logger.error para proveer todo el contexto de lo que sucedió). Otras alternativas podrían ser retornar otro valor, o levantar una excepción diferente. Si usted elige levantar una excepción distinta, incluya la excepción original que gatilló el problema.

#### Incluir la excepción original
Si elegimos levantar una excepción diferente, y cambiamos el mensaje, es recomendable incluir la excepción original que llevó a aquello. Podemos usar el syntax `raise e from original_exception`. Cuando ocupemos esto, el traceback original va a ser incrustada en la nueva excepción, y la excepción original va a ser seteada en el atributo `__cause__` de la nueva excepción resultante.

Siempre utilice el syntaxis `raise e from o` cuando se cambie el tipo de excepción.

#### Usar assertions en Python

assertions son para ser utilizadas en situaciones que no deberían ocurrir nunca, así la expressión en el `assert` statement debe ser una condición imposible. Si esta condición ocurre, significa que hay un defecto en el software. La idea de usar assertions es prevenir que el programa cause más daños si algún escenario inválido se presenta. Un assertion es una condición Booleana en el código que debe ser verdadera para que el programa sea correcto. Assertions no deberían ser utilizados en el control de flujo del programa.

Asegúrese de que el programa termina cuando un assertion falla. Incluya un mensaje descriptivo en el error en el statement de assertion y loggear los errores para asegurarse de que pueda debuggear y corregir el problema después. El siguiente código es una mala idea de usar

In [ ]:
try:
    assert condition.holds(), "Condition is not satisfied"
except AssertionError:
    alternative_procedure()

Una mejor alternativa requiere menos líneas de código, pero provee información más útil.

In [ ]:
result = condition.holds()
assert result > 0, f"Error with {result}"

Cuando use assertions, intente evitar llamados directos a funciones y escriba la expresión en términos de variables locales.

En general, las excepciones son para manejar situaciones inesperadas relacionadas con la lógica que nuestro programa quiere considerar, mientras que assertions son mecanismos de self-checking puestos en el código, para validar (assert) *correctness*.

## Separación de preocupaciones (concerns)

Este es un principio de diseño que es aplicado en múltiples niveles. Diferentes responsabilidades deberían ir en distintos componentes, layers, o módulos de aplicación. Cada parte del programa debería ser responsable de una parte de la funcionalidad (lo que lo llamamos sus *concerns*) y no debería de saber nada del resto.

El objetivo de separar los *concerns* en software es mejorar la mantenibilidad minimizando los *ripple effects*. Un *ripple effect* significa la propagación de un cambio en el software desde un punto de partida.

## Cohesión y coupling

cohesion significa que los objetos deberían tener un propósito pequeño y bien definido, y deberían de hacer tan poco como sea posible.

coupling se refiere a la idea de como dos o más objetos dependen del otro. Esta dependencia posee una limitación. Si dos partes del código son muy dependientes del otro, puede traer consecuencias no deseadas:
- **No reutilización del código:**  Si una función depende mucho de un objeto en particular o toma muchos parámetros, esta función va a ser muy difícil de utilizar en otro contexto.
- **Ripple effects**: Los cambios en alguna de las dos partes van a impactar el otro.
- **Bajo nivel de abstracción**

Rule of thumb: Software bien definido va a lograr alta cohesión y bajo coupling.

## Acrónimos para vivir

### DRY/OAOO
Las ideas *Don't Repeat Yourself (DRY)* y *Once and Only Once (OAOO)* están muy relacionadas. Se debería de evitar la duplicación a toda costa.

Las cosas en el código, conocimiento, tienen que ser definidas una vez y en una sola parte. Cuando se tiene que hacer un cambio en el código, debería de haber un solo lugar en el cual modificar las cosas.

Consecuencias negativas de duplicación de código:
- **Propenso a errores**: Cuando cierta lógica está repetida múltiples veces a través del código, y algo necesita cambiar, significa que dependemos de corregir eficientemente todas las instancias con esta lógica.
- **Es caro:** Hacer un cambio en múltiples lugares toma más tiempo.
- **No es confiable:** Cuando múltiples lugares necesitan ser modificados por un solo cambio en el contexto, se confía en que la persona que escribió el código pueda recordar todas las instancias donde la modificación tiene que hacerse.

El acercamiento más simple para eliminar la duplicación de código es crear una función

### YAGNI

*You ain't gonna need it* es una idea que deberías mantener en mente a menudo cuando se escriba una solución si no se le quiere hacer *over-engineering*. Queremos ser capaces de modificar fácilmente nuestros programas, por lo que los queremos hacer *future-proof*.

Tener software mantenible no es acerca de anticipar requerimientos futuros. Es acerca de escribir código que solo se dirige en requerimientos actuales de tal manera que va a ser posible (y fácil) de cambiar en un futuro.

### KIS

*Keep it simple*. Implemente funcionalidades mínimas que resuelven correctamente el problema y no complican la solución más de lo necesario. Recordar que mientras más simple sea el diseño, va a ser más mantenible.

### EAFP/LBYL

*Easier to ask for forgiveness than permission (EAFP)* y *Look before you leap (LBYL)*. La idea de EAFP es que escribamos nuestro código para que ejecute una acción directamente y luego nos preocupemos de las consecuencias en caso de que no funcione. Típicamente, esto significa correr algún código, esperando que funcione, pero atrapar la excepción si no lo hace y luego manejar el código correctivo en el bloque de except.

Lo anterior es contrario a lo de LBYL, aquí primero se chequea lo que vamos a utilizar.

## Herencia en Python

Si bien la herencia es un concepto poderoso, viene con sus peligros. El principal es que cada vez que extendemos una *base class*, creamos un nuevo objeto que está estrechamente *coupled* con el *parent*.

Uno de los principales escenarios en los que los desarrolladores cuentan con herencia es para la reutilización de código. Si bien deberíamos reutilizar código cuando sea posible, no es una buena idea forzar nuestro diseño para que use herencia para reutilizar código solo porque podemos obtener los métodos de la *parent class* gratis. La forma correcta de reutilizar código es tener objetos altamente cohesivos que pueden ser fácilmente compuestos y pueden funcionar en múltiples contextos.

## Cuando la herencia es una buena decisión

Cuando creemos una nueva subclase, tenemos que pensar si realmente va a utilizar todos los métodos que heredo, como forma heurística para ver si la clase está correctamente definida. Si encontramos que no necesitamos la mayoría de los métodos y los tenemos que reemplazar o hacer override, puede ser un error de diseño que podría ser causado por: 
- La superclass está vagamente definida y contiene mucha responsabilidad, en vez de una interfaz bien definida.
- La subclass no es una especialización apropiada de la superclass que está intentando extender

Un buen caso para usar herencia es el tipo de situación cuando se tiene una clase que define ciertos componentes con su comportamiento que están definidos por la interfaz de esta clase y luego se necesita especializar la clase para así crear objetos que hacen lo mismo pero con algo más añadido, o con algunas partes en particular con su comportamiento cambiado.

Hablando de definición de interfaces, este es otro buen uso para herencia. Cuando queremos hacer cumplir cierta interfaz de algunos objetos, podemos crear una *abstract base class* que no implementa el comportamiento en si misma, pero define una interfaz - cada clase que la extiende va a tener que implementarlos para así ser un subtype apropiado.

Finalmente, otro buen caso para la herencia es en excepciones.

## Antipatrones para herencia

El uso correcto de herencia es para especializar objetos y crear abstracciones más detalladas partiendo de algunas bases.

La base class es parte de la definición pública de la nueva clase derivada. Esto es porque los métodos que son heredados van a ser parte de la interfaz de esta nueva clase. Por esta razón, cuando leamos los métodos públicos de una clase, deben ser consistentes con lo que la base class define.

## Múltiple herencia en Python

La multi-herencia es una espada de doble filo. Puede ser muy beneficiosa en algunos casos. No hay nada de malo en multi-herencia, el único problema es que cuando no se implementa correctamente, va a multiplicar los problemas.

Una de las aplicaciones más poderosas de multi-herencia es probablemente que permite la creación de mixins.

## Method Resolution Order (MRO)

A algunas personas no les gusta la multi-herencia debido a las restricciones que tiene en otros lenguajes de programación, como el llamado problema del diamante. Para resolver esto, Python utiliza un algoritmo llamado C3 Linearization o MRO.

## Mixins

## Estructuración del código

La manera en la que el código está organizado impacta el rendimiento del equipo y su mantenibilidad.

En particular, tener archivos largos con muchas definiciones es una mala práctica y debería ser desanimada. Esto no significa ir al extremo de poner una definición por archivo, pero un buen código va a estructurar y organizar sus componentes según similitud.

La idea sería crear un nuevo directorio con un archivo `__init__.py` (esto lo convertirá en un Python package). Junto a este archivo, podríamos tener múltiples archivos con todas las definiciones en particular que cada uno requiere. Luego `__init__.py` importará de todos estos archivos las definiciones que tienen. Además de que va a ser más fácil navegar por cada archivo, las cosas serán más fáciles de encontrar, podríamos argumentar que esto es más eficiente por las siguientes razones:
- Contiene mejos objetos a *parsear* y cargar en la memoria cuando el módulo es importado
- El módulo en si mismo va a importar menos módulos porque necesita menos dependencias

También nos puede servir que en vez de colocar constantes en todos los archivos, podemos crear un archivo separado para guardar las constantes que serán utilizadas en el proyecto, e importarlas. Centralizar la información de esta manera hace más fácil el poder reutilizar código y ayuda a evitar duplicación inadvertida.